In [ ]:
import json
from langchain.agents import create_json_agent
from langchain_community.agent_toolkits import JsonToolkit
from langchain_community.tools.json.tool import JsonSpec
from langchain_openai import OpenAI
from dotenv import load_dotenv

In [ ]:
load_dotenv(dotenv_path=".env")
with open("defillama_pools.json") as f:
    data = json.load(f)
json_spec = JsonSpec(dict_=data, max_value_length=4000)
json_toolkit = JsonToolkit(spec=json_spec)

json_agent_executor = create_json_agent(
    llm=OpenAI(temperature=0), toolkit=json_toolkit, verbose=True
)

In [ ]:
json_agent_executor.run("Uniswap V3的USDC-WETH池的APY是多少？")

In [ ]:
json_agent_executor.run("binance cex的tvl有多少")

In [ ]:
import requests

url = "https://api.llama.fi/protocols"
headers = {"accept": "*/*"}
response = requests.get(url, headers=headers)
data = response.json()
print("data")

In [ ]:
import requests
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    PromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
)
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain_community.tools.convert_to_openai import format_tool_to_openai_tool
from langchain_openai import ChatOpenAI
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from my_langchain_anthropic.experimental import ChatAnthropicTools
from langchain_core.runnables import ConfigurableField
from anthropic_tools import (
    AnthropicToolsAgentOutputParser,
    format_to_anthropic_tool_messages,
)
from langchain_core.utils.function_calling import convert_to_openai_function

url = "https://api.llama.fi/protocols"
headers = {"accept": "*/*"}
response = requests.get(url, headers=headers)

# 确保请求成功
if response.status_code == 200:
    data = response.json()
    json_data = {}
    for d in data:
        # d["chains"] = d["chainTvls"]
        json_data[d["name"]] = d
    json_spec = JsonSpec(dict_=json_data, max_value_length=4000)
    json_toolkit = JsonToolkit(spec=json_spec)
    tools = json_toolkit.get_tools()
else:
    raise Exception("请求失败，状态码：", response.status_code)

llm_agent = ChatAnthropicTools(
    model="claude-3-opus-20240229",
    # max_tokens=,
    temperature=0.7,
    # anthropic_api_key=os.environ.get("ANTHROPIC_API_KEY", "not_provided"),
    streaming=True,
    verbose=True,
).configurable_alternatives(  # This gives this field an id
    # When configuring the end runnable, we can then use this id to configure this field
    ConfigurableField(id="llm"),
    default_key="anthropic_claude_3_opus",
    openai_gpt_3_5_turbo_1106=ChatOpenAI(
        model="gpt-3.5-turbo-1106",
        verbose=True,
        streaming=True,
        temperature=0.9,
    ),
    openai_gpt_4_turbo_preview=ChatOpenAI(
        temperature=0.9,
        model="gpt-4-turbo-preview",
        verbose=True,
        streaming=True,
    ),
)
system_message = """You are a very useful assistant. 

Please first use json_spec_list_keys(data) to find the relevant keys, and then call json_spec_get_value with the most relevant key.

For example, when json_spec_get_value(data["key1"]) returns a KeyError, please call json_spec_list_keys(data) again to find a key2 related to "key1", and then call json_spec_get_value(data["key2"]) again.

In the JSON file, key `chains` is an array of string. It means the project has tvl on those chains.

And the key 'chainTvls' is a obj, it means the tvl on chains.

`change_1h` represents the change of tvl within 1 hour.
`change_1d` represents the change of tvl within 1 day. 
`change_d` represents the change of tvl within 7 days.
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
).configurable_alternatives(
    which=ConfigurableField("llm"),
    default_key="anthropic_claude_3_opus",
    openai_gpt_4_turbo_preview=ChatPromptTemplate.from_messages(
        [
            SystemMessagePromptTemplate.from_template(template=system_message),
            HumanMessagePromptTemplate.from_template("{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    ),
    openai_gpt_3_5_turbo_1106=ChatPromptTemplate.from_messages(
        [
            SystemMessagePromptTemplate.from_template(template=system_message),
            HumanMessagePromptTemplate.from_template("{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    ),
)
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_anthropic_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    # | prompt_trimmer # See comment above.
    | llm_agent.bind(tools=[convert_to_openai_function(tool) for tool in tools])
    | AnthropicToolsAgentOutputParser()
).configurable_alternatives(
    which=ConfigurableField("llm"),
    default_key="anthropic_claude_3_opus",
    openai_gpt_4_turbo_preview=(
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_to_openai_tool_messages(
                x["intermediate_steps"]
            ),
        }
        | prompt
        # | prompt_trimmer # See comment above.
        | llm_agent.bind(tools=[format_tool_to_openai_tool(tool) for tool in tools])
        | OpenAIToolsAgentOutputParser()
    ),
    openai_gpt_3_5_turbo_1106=(
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_to_openai_tool_messages(
                x["intermediate_steps"]
            ),
        }
        | prompt
        # | prompt_trimmer # See comment above.
        | llm_agent.bind(tools=[format_tool_to_openai_tool(tool) for tool in tools])
        | OpenAIToolsAgentOutputParser()
    ),
)

In [ ]:
agent = agent.with_config({"configurable": {"llm": "openai_gpt_4_turbo_preview"}})
excutor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)
res = excutor.invoke({"input": "uniswap tvl的变化"})
print(res)

In [ ]:
from defillama_wrapper import DefiLLamaWrapTVLPtotocols

tvlWrapper = DefiLLamaWrapTVLPtotocols()


In [ ]:
from langchain.agents import AgentExecutor

# agent = tvlWrapper.create_agent().with_config({"configurable": {"llm": "openai_gpt_4_turbo_preview"}})
agent = tvlWrapper.create_agent()
excutor = AgentExecutor(
    agent=agent,
    tools=tvlWrapper.tools,
    verbose=True,
)
res =  excutor.invoke({"input": "curve tvl的变化","chat_history":[]})
print(res)